In [55]:
import os
import shutil
import random

from PIL import Image, ImageDraw

In [2]:
project_dir = "./data/tweet_parse-2024-09-18-18-04-7d5636ed/"

In [4]:
def save_org_dir(d):
    org_dir = d + "_org"
    if not os.path.exists(org_dir):
        shutil.copytree(d, org_dir)

labels_dir = os.path.join(project_dir, "labels")
images_dir = os.path.join(project_dir, "images")

save_org_dir(labels_dir)
save_org_dir(images_dir)

In [205]:
def get_bottom_cut_labels(labels_f, cut_point, img_height):
    new_labels_data = []
    with open(labels_f, 'r') as infile:
        for line in infile.readlines():
            label_id, x_center, y_center, width, height = line.split()
            y_center, height = float(y_center), float(height)

            # Convert normalized values to actual pixel values
            actual_y_center = y_center * img_height
            actual_height = height * img_height

            # New height of the image after the cut
            new_img_height = cut_point * img_height

            # Check if the bottom of the bounding box is below the cut line
            if actual_y_center + actual_height/2 > new_img_height:
                # Adjust the height of the bounding box if part of it is below the cut
                if actual_y_center - actual_height/2 < new_img_height:
                    new_height = (new_img_height - (actual_y_center - actual_height/2)) / new_img_height
                    new_y_center = (actual_y_center - actual_height/2 + new_height * new_img_height / 2) / new_img_height
                    new_line = f"{label_id} {x_center} {new_y_center} {width} {new_height}\n"
                    new_labels_data.append(new_line)
            else:
                # If the entire bounding box is above the cut, normalize to the new image height
                new_y_center = actual_y_center / new_img_height
                new_height = height * img_height / new_img_height
                new_line = f"{label_id} {x_center} {new_y_center} {width} {new_height}\n"
                new_labels_data.append(new_line)

    return new_labels_data

def get_top_cut_labels(labels_f, cut_point, img_height):
    new_labels_data = []
    with open(labels_f, 'r') as infile:
        for line in infile.readlines():
            label_id, x_center, y_center, width, height = line.split()
            y_center, height = float(y_center), float(height)

            # Convert normalized values to actual pixel values
            actual_y_center = y_center * img_height
            actual_height = height * img_height

            # Calculate the actual top and bottom of the bounding box
            box_bottom = actual_y_center + actual_height / 2
            box_top = actual_y_center - actual_height / 2

            # New height of the image after the cut
            new_img_height = img_height * (1 - cut_point)

            # Adjust labels if at least the bottom of the bounding box is below the cut line
            if box_bottom >= cut_point * img_height:
                # Normalize y_center and height to the new image height
                new_y_center = (actual_y_center - cut_point * img_height) / new_img_height
                new_height = actual_height / new_img_height
                if new_y_center - new_height / 2 < 0:
                    new_y_bottom = new_y_center + new_height / 2
                    new_y_center = new_y_bottom / 2
                    new_height = new_y_bottom
                new_line = f"{label_id} {x_center} {new_y_center} {width} {new_height}\n"
                new_labels_data.append(new_line)

    return new_labels_data



In [274]:
for im_path in os.listdir(images_dir):
    for _ in range(3):
        im = Image.open(os.path.join(images_dir, im_path))
        labels_path = os.path.join(labels_dir, im_path.replace('.jpg', '.txt'))
        cut_point = round(random.uniform(0.3, 0.7), 2)
        bottom_cut = random.randint(0, 1)
        if bottom_cut:
            bottom_y_thresh = im.height * cut_point
            cut_im = im.crop([0, 0, im.width, bottom_y_thresh])
            new_labels_data = get_bottom_cut_labels(labels_path, cut_point=cut_point, img_height=im.height)
        else:
            top_y_thresh = im.height * cut_point
            cut_im = im.crop([0, top_y_thresh, im.width, im.height])
            new_labels_data = get_top_cut_labels(labels_path, cut_point=cut_point, img_height=im.height)
        
        new_im_name = f"{im_path.split('.')[0]}_{cut_point}_{bottom_cut}"
        cut_im.save(os.path.join(images_dir, new_im_name + ".jpg"))
        with open(os.path.join(labels_dir, new_im_name + ".txt"), 'w') as outfile:
            for new_line in new_labels_data:
                outfile.write(new_line)

In [270]:
cut_im_c = cut_im.copy()
draw = ImageDraw.Draw(cut_im_c)
for l in new_labels_data:
    l_s = [float(v) for v in l.strip().split(" ")]
    x_center, y_center, width, height = l_s[1], l_s[2], l_s[3], l_s[4]
    x1 = (x_center - width/2) * cut_im_c.width
    y1 = (y_center - height/2) * cut_im_c.height
    x2 = (x_center + width/2) * cut_im_c.width
    y2 = (y_center + height/2) * cut_im_c.height
    draw.rectangle((x1, y1, x2, y2), outline="red", width=5)

In [271]:
cut_im_c.show()

In [130]:
with open(labels_path, 'r') as infile:
    labels_data = list()
    for line in infile.readlines():
        labels_data.append(line)

In [139]:
im_c = im.copy()
draw = ImageDraw.Draw(im_c)
for l in labels_data:
    l_s = [float(v) for v in l.strip().split(" ")]
    x_center, y_center, width, height = l_s[1], l_s[2], l_s[3], l_s[4]
    x1 = (x_center - width/2) * im_c.width
    y1 = (y_center - height/2) * im_c.height
    x2 = (x_center + width/2) * im_c.width
    y2 = (y_center + height/2) * im_c.height
    draw.rectangle((x1, y1, x2, y2), outline="red", width=5)

In [140]:
im_c.show()

In [179]:
y2

857.6636703490432